# NutriGenius: Food Object Detection - Exploratory Data Analysis

Notebook ini melakukan exploratory data analysis pada dataset gambar makanan untuk membangun model deteksi objek makanan dalam aplikasi NutriGenius.

## Daftar Isi
1. [Pendahuluan](#pendahuluan)
2. [Setup Dataset](#setup-dataset)
3. [Eksplorasi Dataset](#eksplorasi-dataset)
4. [Preprocessing Dataset](#preprocessing-dataset)
5. [Visualisasi Data](#visualisasi-data)
6. [Kesimpulan](#kesimpulan)

## 1. Pendahuluan

Notebook ini mengeksplorasi dataset gambar makanan untuk keperluan pengembangan komponen deteksi objek pada aplikasi NutriGenius. Komponen ini akan mendeteksi jenis makanan dalam foto yang diambil oleh pengguna, yang kemudian akan digunakan untuk memberikan rekomendasi nutrisi dan artikel yang relevan.

Fokus eksplorasi ini mencakup:
- Pemahaman distribusi data berbagai jenis makanan
- Analisis karakteristik visual makanan khas Indonesia dan internasional
- Persiapan data untuk model deteksi objek

## 2. Setup Dataset

Untuk model deteksi makanan, kita akan menggunakan subset dari dataset Food-101 dan menambahkan gambar makanan Indonesia. Kita juga menyediakan dataset yang lebih kecil untuk prototyping cepat.

In [ ]:
import os
import sys
import shutil
import urllib.request
import zipfile
import tarfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import cv2
from PIL import Image
import glob
import logging

# Konfigurasi logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Path konfigurasi
BASE_DIR = os.path.abspath(os.path.join(os.path.dirname("__file__"), '..', '..'))
DATA_DIR = os.path.join(BASE_DIR, "data")
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw")
FOOD_DATA_DIR = os.path.join(RAW_DATA_DIR, "food_images")
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, "processed", "food")

# Buat direktori jika belum ada
for directory in [DATA_DIR, RAW_DATA_DIR, FOOD_DATA_DIR, PROCESSED_DATA_DIR]:
    os.makedirs(directory, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"Food data will be stored in: {FOOD_DATA_DIR}")

### 2.1 Download Dataset

Kita akan menyiapkan dataset gambar makanan dengan dua opsi: subset Food-101 (~50MB) atau beberapa gambar sampel untuk prototyping cepat.

In [ ]:
def download_with_progress(url, dest_path):
    """Download file dengan progress bar"""
    try:
        with tqdm(unit='B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
            urllib.request.urlretrieve(
                url, 
                dest_path, 
                reporthook=lambda b, bsize, tsize: t.update(bsize if tsize == -1 else min(bsize, tsize-t.n))
            )
        return True
    except Exception as e:
        logger.error(f"Gagal mengunduh {url}: {e}")
        return False

def setup_food_dataset(use_larger_dataset=False, include_indonesian_food=True):
    """
    Menyiapkan dataset gambar makanan
    
    Args:
        use_larger_dataset: Jika True, unduh subset Food-101 (~50MB), jika False gunakan sedikit gambar sampel
        include_indonesian_food: Jika True, tambahkan gambar makanan Indonesia
        
    Returns:
        bool: True jika berhasil, False jika gagal
    """
    # Periksa apakah dataset sudah ada
    sample_image_path = os.path.join(FOOD_DATA_DIR, "pizza/1005649.jpg")
    dataset_exists = os.path.exists(sample_image_path) or len(glob.glob(os.path.join(FOOD_DATA_DIR, "*/*.jpg"))) > 0
    
    if dataset_exists:
        logger.info("Dataset gambar makanan sudah ada.")
        return True
    
    success = True
    
    if use_larger_dataset:
        # Food-101 dataset subset (hanya 5 kategori, ~50MB)
        FOOD_URL = "https://github.com/nutrigenius-samples/food101-subset/archive/main.zip"
        FOOD_ZIP = os.path.join(DATA_DIR, "food101_subset.zip")
        
        logger.info("Mengunduh subset dataset Food-101 (~50MB). Proses ini mungkin membutuhkan waktu...")
        success = download_with_progress(FOOD_URL, FOOD_ZIP)
        
        if not success:
            logger.warning("Gagal mengunduh subset Food-101. Mencoba sampel kecil...")
            return setup_food_dataset(use_larger_dataset=False, include_indonesian_food=include_indonesian_food)
        
        # Ekstrak dataset
        try:
            logger.info("Mengekstrak subset dataset Food-101...")
            with zipfile.ZipFile(FOOD_ZIP, 'r') as zip_ref:
                zip_ref.extractall(DATA_DIR)
            
            # Pindahkan file ke lokasi yang benar
            food_subset_dir = os.path.join(DATA_DIR, "food101-subset-main", "images")
            for category in os.listdir(food_subset_dir):
                category_dir = os.path.join(food_subset_dir, category)
                if os.path.isdir(category_dir):
                    os.makedirs(os.path.join(FOOD_DATA_DIR, category), exist_ok=True)
                    files_to_copy = os.listdir(category_dir)[:min(100, len(os.listdir(category_dir)))]  # Ambil maks 100 gambar per kategori
                    for file in files_to_copy:
                        shutil.copy(
                            os.path.join(category_dir, file),
                            os.path.join(FOOD_DATA_DIR, category, file)
                        )
            
            # Bersihkan
            os.remove(FOOD_ZIP)
            shutil.rmtree(os.path.join(DATA_DIR, "food101-subset-main"))
            
            logger.info("Setup subset dataset Food-101 berhasil!")
        except Exception as e:
            logger.error(f"Gagal mengekstrak subset dataset Food-101: {e}")
            success = False
    else:
        # Hanya beberapa gambar sampel (~1MB)
        # Buat folder kategori
        categories = ["pizza", "banana", "apple", "salad", "rice"]
        for category in categories:
            os.makedirs(os.path.join(FOOD_DATA_DIR, category), exist_ok=True)
        
        # Sampel gambar makanan
        SAMPLE_FOOD_URLS = [
            ("https://upload.wikimedia.org/wikipedia/commons/a/a3/Eq_it-na_pizza-margherita_sep2005_sml.jpg", "pizza/pizza_sample1.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/d/d1/Pepperoni_pizza.jpg", "pizza/pizza_sample2.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/e/e8/Bananas.jpg", "banana/banana_sample1.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/8/8a/Banana-Single.jpg", "banana/banana_sample2.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/c/ce/Red_Apple.jpg", "apple/apple_sample1.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/5/5b/Äpple_ett_rött.jpg", "apple/apple_sample2.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/9/94/Salad_platter.jpg", "salad/salad_sample1.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/f/f3/VegetableSaladLunchPlate.jpg", "salad/salad_sample2.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/8/81/Bowl_of_rice.jpg", "rice/rice_sample1.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/7/7b/White_rice.JPG", "rice/rice_sample2.jpg"),
        ]
        
        logger.info("Mengunduh gambar sampel makanan...")
        for url, rel_path in SAMPLE_FOOD_URLS:
            success = success and download_with_progress(url, os.path.join(FOOD_DATA_DIR, rel_path))
        
        if not success:
            logger.error("Gagal mengunduh semua gambar sampel makanan.")
        else:
            logger.info("Gambar sampel makanan berhasil diunduh!")
    
    # Tambahkan makanan Indonesia jika diminta
    if include_indonesian_food and success:
        indonesian_food_dir = os.path.join(FOOD_DATA_DIR, "indonesian_food")
        os.makedirs(indonesian_food_dir, exist_ok=True)
        
        INDONESIAN_FOOD_URLS = [
            ("https://upload.wikimedia.org/wikipedia/commons/3/30/Nasi_Goreng_rumahan.jpg", "nasi_goreng.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/0/01/Sate_Ponorogo.jpg", "sate.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/d/d2/Gado-gado_1.JPG", "gado_gado.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/3/36/Soto_Betawi.jpg", "soto.jpg"),
            ("https://upload.wikimedia.org/wikipedia/commons/6/65/Rendang_Padang.jpg", "rendang.jpg"),
        ]
        
        logger.info("Mengunduh gambar makanan Indonesia...")
        for url, filename in INDONESIAN_FOOD_URLS:
            success = success and download_with_progress(url, os.path.join(indonesian_food_dir, filename))
        
        if not success:
            logger.warning("Gagal mengunduh beberapa gambar makanan Indonesia, tetapi melanjutkan dengan yang berhasil...")
        else:
            logger.info("Gambar makanan Indonesia berhasil diunduh!")
    
    return success

# Unduh dataset (default: dataset kecil)
USE_LARGER_DATASET = False  # Ubah menjadi True untuk dataset yang lebih besar
INCLUDE_INDONESIAN_FOOD = True  # Ubah menjadi False untuk mengecualikan gambar makanan Indonesia
success = setup_food_dataset(USE_LARGER_DATASET, INCLUDE_INDONESIAN_FOOD)

if success:
    print("✅ Dataset gambar makanan siap digunakan!")
    # Hitung jumlah file untuk konfirmasi
    total_images = 0
    categories = [d for d in os.listdir(FOOD_DATA_DIR) if os.path.isdir(os.path.join(FOOD_DATA_DIR, d))]
    for category in categories:
        num_images = len(glob.glob(os.path.join(FOOD_DATA_DIR, category, "*.jpg")))
        print(f"   - {category}: {num_images} gambar")
        total_images += num_images
    print(f"   Total: {total_images} gambar dalam {len(categories)} kategori")
else:
    print("❌ Gagal menyiapkan dataset makanan. Periksa log untuk detail.")
    print("   Akan mencoba melanjutkan dengan gambar sampel yang tersedia.")

### 2.2 Struktur File/Folder yang Diharapkan

Dataset gambar makanan disimpan dengan struktur file sebagai berikut:

```
NutriGenius/
├── data/
│   ├── raw/
│   │   └── food_images/           # Dataset gambar makanan disimpan di sini
│   │       ├── pizza/             # Setiap kategori dalam folder terpisah
│   │       │   ├── 1005649.jpg
│   │       │   ├── 1011328.jpg
│   │       │   └── ...
│   │       ├── banana/
│   │       ├── apple/
│   │       ├── salad/
│   │       ├── rice/
│   │       └── indonesian_food/   # Folder khusus makanan Indonesia
│   └── processed/
│       └── food/                  # Dataset yang telah diproses
│           ├── train/
│           ├── val/
│           └── test/
```

### 2.3 Verifikasi Dataset

Mari kita periksa apakah dataset sudah tersedia dan valid:

In [ ]:
def verify_food_dataset():
    """Verifikasi dataset gambar makanan"""
    # Cek apakah directory ada
    if not os.path.exists(FOOD_DATA_DIR):
        logger.error(f"Directory dataset gambar makanan tidak ditemukan: {FOOD_DATA_DIR}")
        return False
    
    # Cek apakah ada folder kategori
    categories = [d for d in os.listdir(FOOD_DATA_DIR) if os.path.isdir(os.path.join(FOOD_DATA_DIR, d))]
    if len(categories) == 0:
        logger.error("Tidak ada folder kategori makanan ditemukan")
        return False
    
    # Cek apakah ada file gambar di setiap kategori
    valid_categories = 0
    for category in categories:
        image_files = glob.glob(os.path.join(FOOD_DATA_DIR, category, "*.jpg"))
        if len(image_files) > 0:
            # Coba buka satu gambar untuk memastikan format valid
            try:
                img = Image.open(image_files[0])
                img.close()
                valid_categories += 1
            except:
                logger.warning(f"Tidak dapat membuka gambar dalam kategori {category}")
    
    if valid_categories > 0:
        logger.info(f"Dataset valid dengan {valid_categories} kategori")
        return True
    else:
        logger.error("Tidak ada kategori valid dalam dataset")
        return False

# Verifikasi dataset
is_valid = verify_food_dataset()
if not is_valid:
    print("⚠️ Dataset tidak valid atau tidak lengkap.")
    print("   Silakan unduh dataset secara manual atau jalankan kode setup_food_dataset() lagi.")
    
    # Fallback: buat minimal dataset jika perlu
    if len(glob.glob(os.path.join(FOOD_DATA_DIR, "*/*.jpg"))) == 0:
        print("   Membuat minimal dataset fallback...")
        # Buat folder fallback
        fallback_dir = os.path.join(FOOD_DATA_DIR, "fallback")
        os.makedirs(fallback_dir, exist_ok=True)
        
        # Unduh 1 gambar fallback
        fallback_url = "https://upload.wikimedia.org/wikipedia/commons/6/6d/Good_Food_Display_-_NCI_Visuals_Online.jpg"
        fallback_path = os.path.join(fallback_dir, "food_sample.jpg")
        download_with_progress(fallback_url, fallback_path)
        print("   Dataset fallback minimal dibuat. Performa model akan terbatas.")

## 3. Eksplorasi Dataset

// ... existing code ...